In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

import holidays
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

from datetime import datetime, timedelta



In [ ]:
# df = df.rename(columns={
#     "id_compteur": "counter_id",
#     "nom_compteur": "counter_name",
#     "id": "site_id",
#     "name": "site_name",
#     "date": "date",
#     "installation_date": "counter_installation_date",
#     "coordinates": "coordinates",
#     "counter": "counter_technical_id"
# })

# columns_to_keep = [
#     "counter_id", "counter_name", "site_id", "site_name", "date",
#     "counter_installation_date", "coordinates", "counter_technical_id"
# ]
# df = df[columns_to_keep]

In [ ]:
# # Drop rows where the 'coordinates' column is NaN
# df = df.dropna(subset=['coordinates'])

# # Reset index after dropping rows
# df = df.reset_index(drop=True)

# # Display the updated DataFrame
# df.head(2)

In [ ]:
# import struct

# # Function to decode latitude and longitude from byte string
# def decode_coordinates(byte_string):
#     try:
#         header_size = 5  # Skip the first 5 bytes as a header
#         lat, lon = struct.unpack("dd", byte_string[header_size:])
#         return lat, lon
#     except Exception as e:
#         print(f"Error decoding: {e}")
#         return None, None

# # Apply the function to the DataFrame
# df["latitude"], df["longitude"] = zip(*df["coordinates"].apply(decode_coordinates))

# # Drop the original 'coordinates' column if no longer needed
# df = df.drop(columns=["coordinates"])

# # Display the updated DataFrame
# df.head(2)


In [ ]:
# # Ensure the 'date' column is in datetime format
# df['date'] = pd.to_datetime(df['date'])

# # Format the 'date' column to 'YYYY-MM-DD HH:MM:SS'
# df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# # Display the updated DataFrame
# df.head(2)

In [ ]:
# # Ensure the 'date' column is in datetime format
# df['date'] = pd.to_datetime(df['date'])

# # Find the earliest and latest dates
# earliest_date = df['date'].min()
# latest_date = df['date'].max()

# print(f"Earliest date: {earliest_date}")
# print(f"Latest date: {latest_date}")


Earliest date: 2022-03-27 23:00:00
Latest date: 2024-12-07 23:00:00


In [ ]:
df1 = pd.read_csv(r'data\extra\2019_comptage-velo-donnees-compteurs.csv\2019_comptage-velo-donnees-compteurs-2.csv', delimiter=';')
df2 = pd.read_csv(r'data\extra\2020_comptage-velo-donnees-compteurs.csv\comptage-velo-donnees-compteurs.csv', delimiter=';')
df3 = pd.read_csv(r'data\extra\2021-comptage-velo-donnees-compteurs.csv\comptage-velo-donnees-compteurs.csv', delimiter=';')
df4 = pd.read_csv(r'data\extra\2022-comptage-velo-donnees-compteurs\comptage-velo-donnees-compteurs.csv', delimiter=';')


C:\Users\nishu\AppData\Local\Temp\ipykernel_20828\549738711.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv(r'data\extra\2021-comptage-velo-donnees-compteurs.csv\comptage-velo-donnees-compteurs.csv', delimiter=';')


In [86]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)


In [87]:
df.tail()

,Identifiant du compteur,Nom du compteur,Identifiant du site de comptage,Nom du site de comptage,Comptage horaire,Date et heure de comptage,Date d'installation du site de comptage,Lien vers photo du site de comptage,Coordonnées géographiques
4155663,353326452-100047541,Pont National SO-NE,100047541,Pont National,0.0,2022-12-31T19:00:00,22-08-03,https://filer.eco-counter-tools.com/file/2a/79...,"48.82639,2.38448"
4155664,353326452-100047541,Pont National SO-NE,100047541,Pont National,0.0,2022-12-31T20:00:00,22-08-03,https://filer.eco-counter-tools.com/file/2a/79...,"48.82639,2.38448"
4155665,353326452-100047541,Pont National SO-NE,100047541,Pont National,0.0,2022-12-31T21:00:00,22-08-03,https://filer.eco-counter-tools.com/file/2a/79...,"48.82639,2.38448"
4155666,353326452-100047541,Pont National SO-NE,100047541,Pont National,0.0,2022-12-31T22:00:00,22-08-03,https://filer.eco-counter-tools.com/file/2a/79...,"48.82639,2.38448"
4155667,353326452-100047541,Pont National SO-NE,100047541,Pont National,0.0,2022-12-31T23:00:00,22-08-03,https://filer.eco-counter-tools.com/file/2a/79...,"48.82639,2.38448"


In [96]:
# Rename columns to match the desired format
df = df.rename(columns={
    "Identifiant du compteur": "counter_id",
    "Nom du compteur": "counter_name",
    "Identifiant du site de comptage": "site_id",
    "Nom du site de comptage": "site_name",
    "Date et heure de comptage": "date",
    "Date d'installation du site de comptage": "counter_installation_date",
    "Coordonnées géographiques": "coordinates",
    "Comptage horaire": "result"
})


# Split 'coordinates' into 'latitude' and 'longitude'
def extract_lat_long(coords):
    if isinstance(coords, str) and ',' in coords:
        lat, long = coords.split(",")
        return float(lat), float(long)
    return None, None

df["latitude"], df["longitude"] = zip(*df["coordinates"].apply(extract_lat_long))

# Drop the 'coordinates' column as it is no longer needed
# combined_df = combined_df.drop(columns=["coordinates"])

# Rearrange columns to match the desired format
df = df[[
    "counter_id", "counter_name", "site_id", "site_name", "date",
    "counter_installation_date", "latitude", "longitude", "result"
]]

df.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,latitude,longitude,result
0,100047539-SC,21 Boulevard Auguste Blanqui SO-NE,100047539,21 Boulevard Auguste Blanqui SO-NE,2019-01-01 07:00:00,2018-12-05,48.830449,2.353199,0.0
1,100047539-SC,21 Boulevard Auguste Blanqui SO-NE,100047539,21 Boulevard Auguste Blanqui SO-NE,2019-01-02 18:00:00,2018-12-05,48.830449,2.353199,31.0
2,100047539-SC,21 Boulevard Auguste Blanqui SO-NE,100047539,21 Boulevard Auguste Blanqui SO-NE,2019-01-03 16:00:00,2018-12-05,48.830449,2.353199,24.0
3,100047539-SC,21 Boulevard Auguste Blanqui SO-NE,100047539,21 Boulevard Auguste Blanqui SO-NE,2019-01-04 04:00:00,2018-12-05,48.830449,2.353199,2.0
4,100047539-SC,21 Boulevard Auguste Blanqui SO-NE,100047539,21 Boulevard Auguste Blanqui SO-NE,2019-01-04 08:00:00,2018-12-05,48.830449,2.353199,0.0


In [97]:
df['date'] = df['date'].astype(str).apply(lambda x : x.split('+')[0])
df['date'] = df['date'].replace('+', ' ')
df['date'] = pd.to_datetime(df['date'])

In [98]:
df.isna().sum()

counter_id                        0
counter_name                      0
site_id                           0
site_name                         0
date                              0
counter_installation_date    376680
latitude                          0
longitude                         0
result                            0
dtype: int64

In [102]:
y = np.log1p(df['result'].values)

In [103]:
def _encode_dates(X):
    X = X.copy()  # Modify a copy of X
    
    # Ensure 'date' is in datetime format
    X["date"] = pd.to_datetime(X["date"])
    
    # Extract date components
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # Identify weekends (Saturday = 5, Sunday = 6)
    X["is_weekend"] = X["weekday"].isin([5, 6])
    
    # Get French holidays for all years in the dataset
    years = X["year"].unique()
    fr_holidays = holidays.France(years=years)
    
    # Identify holidays
    X["is_holiday"] = X["date"].dt.date.isin(fr_holidays)
    
    # Drop the original 'date' column
    return X


In [104]:
date_encoder = FunctionTransformer(_encode_dates, validate=False)
df = date_encoder.fit_transform(df)
df.head(2)

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,latitude,longitude,result,year,month,day,weekday,hour,is_weekend,is_holiday
0,100047539-SC,21 Boulevard Auguste Blanqui SO-NE,100047539,21 Boulevard Auguste Blanqui SO-NE,2019-01-01 07:00:00,2018-12-05,48.830449,2.353199,0.0,2019,1,1,1,7,False,True
1,100047539-SC,21 Boulevard Auguste Blanqui SO-NE,100047539,21 Boulevard Auguste Blanqui SO-NE,2019-01-02 18:00:00,2018-12-05,48.830449,2.353199,31.0,2019,1,2,2,18,False,False


In [105]:
def train_test_split_temporal(X, y, delta_threshold="30 days"):
    
    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = (X["date"] <= cutoff_date)
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train, y_train, X_valid, y_valid

In [106]:
X = df.copy()

In [ ]:
# Step 1: Preprocessing
# One-hot encode the categorical variables
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
onehot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
categorical_encoded = onehot_encoder.fit_transform(X[categorical_cols])

# Numerical scaling
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(X[numerical_cols])

# Drop processed columns
X.drop(categorical_cols, axis=1, inplace=True)
X.drop(numerical_cols, axis=1, inplace=True)

# Ensure date column is in datetime format
X["date"] = pd.to_datetime(X["date"])

# Combine all features
X_combined = np.hstack([X.values, categorical_encoded, numerical_scaled])

# Assuming each sample has a single timestep

# Step 3: Temporal Train-Test Split
# Convert X_reshaped back into a DataFrame to preserve the date column
X_combined_df = pd.DataFrame(X_combined, columns=[f"feature_{i}" for i in range(X_combined.shape[1])])
X_combined_df["date"] = X["date"].values  # Restore the date column


# Apply temporal train-test split
X_train_split, y_train_split, X_test_split, y_test_split = train_test_split_temporal(X_combined_df, y)


# Remove the 'date' column after splitting
datetime_columns = X_test_split.select_dtypes(include=['datetime64[ns]']).columns
print(f"Columns with datetime64[ns] dtype: {datetime_columns.tolist()}")
datetime_columns = X_test_split.select_dtypes(include=['datetime64[ns]']).columns
print(f"Columns with datetime64[ns] dtype: {datetime_columns.tolist()}")

# Drop these columns from X_train_split
X_train_split = X_train_split.drop(columns=datetime_columns)
# Drop these columns from X_test_split
X_test_split = X_test_split.drop(columns=datetime_columns)

X_train_split = X_train_split.astype(float)
X_test_split = X_test_split.astype(float)



In [ ]:
import utils

final_test = utils.get_test_data()

final_test.head(2)


In [ ]:
date_encoder = FunctionTransformer(_encode_dates, validate=False)
final_test = date_encoder.fit_transform(final_test)
# final_test = pd.get_dummies(final_test, columns=["hour"], prefix="hour")
# final_test.head(2)

# Step 1: Preprocessing
# One-hot encode the categorical variables

categorical_cols = final_test.select_dtypes(include=['object', 'category']).columns
onehot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
categorical_encoded = onehot_encoder.fit_transform(final_test[categorical_cols])

# Numerical scaling
numerical_cols = final_test.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(final_test[numerical_cols])

# Drop processed columns
final_test.drop(categorical_cols, axis=1, inplace=True)
final_test.drop(numerical_cols, axis=1, inplace=True)

# Ensure date column is in datetime format
final_test["date"] = pd.to_datetime(final_test["date"])

# Combine all features
final_test_combined = np.hstack([final_test.values, categorical_encoded, numerical_scaled])

# Assuming each sample has a single timestep

# Convert X_reshaped back into a DataFrame to preserve the date column
final_test_combined = pd.DataFrame(final_test_combined, columns=[f"feature_{i}" for i in range(final_test_combined.shape[1])])
final_test_combined["date"] = final_test["date"].values  # Restore the date column


datetime_columns = final_test_combined.select_dtypes(include=['datetime64[ns]']).columns
print(f"Columns with datetime64[ns] dtype: {datetime_columns.tolist()}")

final_test_combined = final_test_combined.drop(columns=datetime_columns)

final_test_combined = final_test_combined.astype(float)